# ATS Application Tracking System


The aim of this project is to expand a system that analyzes resumes (CVs) uploaded in PDF and DOC formats against Applicant Tracking System (ATS) criteria and provides candidates with feedback on their resume compatibility.

*  The system evaluates CVs against ATS capabilities, identifies missing or weak candidates, and offers appropriate recommendations.  
*  This allows users to enhance their ATS-compliant resumes, making them more visible on job applications and gaining an advantage in the hiring process. 

The project is designed specifically for recent graduates, job seekers, and professionals seeking career advancement.


Stage 1: Basic Data Extraction (CV Parsing)

In [40]:
# Uploading Relevant Libraries
import fitz # Imported name of PyMuPDF library

def extract_text_from_pdf(pdf_path):
    """
    extracts text from PDF file.
    
    Args:
        pdf_path (str): PDF file path.
    
    Returns:
        str: extracting text from PDF.
    """
    try:
        document = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(document)):
            # extract text from the page
            page = document.load_page(page_num)
            text += page.get_text()
        
        # Clear spaces and rotate text
        return " ".join(text.split())

    except Exception as e:
        return f"Hata oluştu: {e}"


# Örnek kullanım (kendi CV'nizin yolunu belirtin)
cv_file = "C:/Users/ilyda/OneDrive/Belgeler/Masaüstü/İlayda Uçan -CV Project Management &business analyst.pdf"
extracted_text = extract_text_from_pdf(cv_file)
if "Hata" not in extracted_text:
  print("Çıkarılan metin:")
  print("-" * 20)
  print(extracted_text)
else:
  print(extracted_text)
"""
"""

Çıkarılan metin:
--------------------
İLAYDA UÇAN LinkedIn Profile | GitHub Profile| ildya.ucna@gmail.com | +90-531-462-6902 | İstanbul/Turkey Professional Summary Results-oriented MIS student with strong experience in project management, stakeholder communication, and digital transformation. Proven leadership in managing 250+ volunteers, applying Agile methodologies, and driving measurable results in organizational planning and customer experience projects. Skilled in project documentation, reporting, and business analysis. Committed to delivering efficiency and alignment in dynamic environments. PROFESSIONAL EXPERIENCE SCA Social – Data & Project Management Intern Istanbul, Turkey | 2025 – Present Designed an organizational structure and management plan for a hypothetical company, implementing project management methodologies. Developed a Project Initiation Document (PID) and Gantt chart for a simulated AI-supported business project, including building predictive models in Python to 

'\n'

Stage 2: Keyword Matching and Scoring Logic


In [41]:
import re

def calculate_keyword_score(cv_text, job_description_text):
    """
    Compares the CV text with keywords extracted from the job description and creates a scoring system.

    Args:
    cv_text (str): Text extracted from the CV.
    job_description_text (str): Text of the job description.

    Returns:
    dict: A dictionary containing matching keywords and scores.
    """
# 1. Extracting keywords from the job description.
    # Let's try to find skill, title, and tool names using Regex.
    # This is a simple start, but it can be expanded later.
    keywords = set(re.findall(r'\b[A-Z][a-zA-Z0-9-]+\b|\b\w+\s\w+\b', job_description_text))

    # Make matching easier by converting words to lowercase
    keywords = {keyword.lower() for keyword in keywords}
    
# 2. Counting how many times the words appear in the CV
    cv_text_lower = cv_text.lower()
    score = {}
    total_matches = 0
    
    for keyword in keywords:
        # Word limit
        matches = len(re.findall(r'\b' + re.escape(keyword) + r'\b', cv_text_lower))
        if matches > 0:
            score[keyword] = matches
            total_matches += matches

# 3. Create the scoring logic
    # For a simple scoring, we can use the total number of matches.
    # For a more advanced scoring, weighting can be used.
    keyword_score_percentage = 0
    if len(keywords) > 0:
        keyword_score_percentage = (total_matches / len(keywords)) * 10
    
    return {
        "matching_keywords": score,
        "total_keywords_in_job_desc": len(keywords),
        "total_matches_in_cv": total_matches,
        "score_percentage": min(keyword_score_percentage, 100) # Max 100 point
    }

# Example usage:
# Text extracted from a CV
sample_cv_text = """
Name: İlayda Uçan
Skills: Python, Java, SQL
Experience: Project Management, Business Analysis
"""
# Text from a job posting
job_ad_text = """ The candidate we are looking for is a business analyst with Python and SQL skills and project management experience.
Proficiency in languages ​​such as Java and C++ is also preferred."""


# Let's run the function
analysis_results = calculate_keyword_score(sample_cv_text, job_ad_text)

# Let's print the results
print("### Keyword Analysis Results ###")
print(f"Matching Percentage: {analysis_results['score_percentage']:.2f}%")
print("---------------------------------------")
print("Matching Keywords:")
for keyword, count in analysis_results['matching_keywords'].items():
    print(f"- '{keyword}' found {count} times")

print(f"\nTotal Keywords in Job Posting: {analysis_results['total_keywords_in_job_desc']}")
print(f"Total Number of Matches in CV: {analysis_results['total_matches_in_cv']}")

### Keyword Analysis Results ###
Matching Percentage: 1.88%
---------------------------------------
Matching Keywords:
- 'python' found 1 times
- 'project management' found 1 times
- 'java' found 1 times

Total Keywords in Job Posting: 16
Total Number of Matches in CV: 3


Stage 3: Advanced ATS Criteria Analysis


In [42]:
def advanced_ats_check(cv_text):
  """
  Analyzes the CV text against advanced ATS criteria and offers suggestions for improvement.

  Args:
  cv_text (str): The text extracted from the CV.

  Returns:
  list: A list containing suggestions for improvement.
  """

  suggestions = []
  cv_text_lower = cv_text.lower()

# 1. Check Standard Section Headers
# Identify the most common standard headers.
  standard_headers = ["experience", "education", "skills", "projects", "contact", "summary"]

  found_headers = [header for header in standard_headers if header in cv_text_lower]

# For example, warn if 'experience' or 'skills' are missing.
  if "experience" not in found_headers and "experience" not in cv_text_lower:
    suggestions.append("No standard headings like 'Experience' or 'Applications' were found in your CV. Please use standard headings so ATSs can easily scan them.")
  if "skills" not in found_headers and "talents" not in cv_text_lower:
    suggestions.append("No standard headings like 'Skills' or 'Abilities' were found in your CV. Use standard headings so your skills are easily recognizable.")

# We can expand this section further. For example:

  if len(found_headers) < 3:

    suggestions.append("Your CV doesn't have enough standard section headings that the ATS can easily recognize. Add headings like 'Education', 'Experience', 'Skills'.")

# 2. Reverse Chronological Sorting Check (Simple Version)

# Use a simple regex to find dates

  years = re.findall(r'\b(19|20)\d{2}\b', cv_text)

  years = [int(year) for year in years]

  if len(years) > 1 and years != sorted(years, reverse=True):

    suggestions.append("Experience and education information may not be listed in reverse chronological order (newest to oldest). This order is very important for ATSs.")

# For a more advanced check, you can check the dates under the headings.

# For example: 'experience' Find the title, check the first date below it.

  return suggestions

# Example usage:
# We get the analysis_results from stage 2.
# We get the cv_text from stage 1.

# Let's get the advanced analysis results.
advanced_suggestions = advanced_ats_check(sample_cv_text)

print("\n### Advanced ATS Analysis and Improvement Suggestions ###")
if not advanced_suggestions:
  print("Great! Your CV appears to adhere to the basic ATS structural rules.")
else:
  for suggestion in advanced_suggestions:
    print(f"- {suggestion}")


### Advanced ATS Analysis and Improvement Suggestions ###
- Your CV doesn't have enough standard section headings that the ATS can easily recognize. Add headings like 'Education', 'Experience', 'Skills'.


Stage 4: AI-Based Improvement Suggestions


In [43]:
import re

def analyze_cv_for_impact(cv_text):
    """
    Analyzes CV text for impact-oriented statements and quantitative data.

    Args:
        cv_text (str): The text extracted from the CV.

    Returns:
        dict: A dictionary containing suggestions regarding impact-oriented phrases and quantitative data.
    """
    suggestions = []
    
    # 1. Impact-Oriented Analysis (Simple Keyword List Method)
    passive_words = ["was responsible for", "was assigned to", "assisted with", "participated in"]
    active_words = ["initiated", "managed", "developed", "achieved", "optimized", "increased", "reduced"]
    
    # Count the words
    passive_count = sum(1 for word in passive_words if word in cv_text.lower())
    active_count = sum(1 for word in active_words if word in cv_text.lower())
    
    # Suggest if passive words outnumber active words
    if passive_count > active_count:
        suggestions.append("Your CV's wording consists of more passive and general sentences. Make it more effective by defining your achievements with action-oriented verbs like 'initiated' or 'managed'.")
    
    # 2. Quantitative Data Deficiency Analysis
    # Search for data like percentages, currency, numerical values, and time periods
    quantitative_data_regex = re.compile(r'(\d+%|\$\d+|\d+k|\d+M|\d+\.\d+|\d+ (month|year|day)s?)')
    
    # Find the number of matches
    matches = quantitative_data_regex.findall(cv_text)
    
    if len(matches) < 3:  # Suggest if there are fewer than 3 quantitative data points
        suggestions.append("Include more quantitative data (numbers, percentages, currency, etc.) to support your concrete achievements. For example, instead of 'I managed projects,' use 'I completed projects 15% ahead of schedule.'")
    
    # Organize the results
    return {
        "suggestions": suggestions,
        "active_word_count": active_count,
        "passive_word_count": passive_count,
        "quantitative_data_found": len(matches)
    }

# Example usage:
# Text extracted from a CV
sample_cv_text_for_ai = """
I was responsible for joining the project management team and managing meetings.
I reduced costs by 15% in 2023 and completed the project in 3 months.
"""
# Run the function
ai_analysis_results = analyze_cv_for_impact(sample_cv_text_for_ai)

print("\n### AI-Powered Feedback and Improvement Suggestions ###")
if not ai_analysis_results['suggestions']:
    print("Great! Your CV seems to have both impact-oriented phrases and quantitative data.")
else:
    for suggestion in ai_analysis_results['suggestions']:
        print(f"- {suggestion}")


### AI-Powered Feedback and Improvement Suggestions ###
- Include more quantitative data (numbers, percentages, currency, etc.) to support your concrete achievements. For example, instead of 'I managed projects,' use 'I completed projects 15% ahead of schedule.'


Stage 5: User Interface (GUI) Development

In [46]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
from tkinter import messagebox
import os
import re
import fitz
import docx

# ----- Tüm yardımcı fonksiyonlar buraya gelecek -----
# Önceki aşamalarda yazdığınız tüm fonksiyonları (Aşama 1'den 4'e kadar) buraya kopyalayıp yapıştırın.
# extract_text_from_pdf()
# extract_text_from_docx()
# extract_text_from_file()
# calculate_keyword_score()
# advanced_ats_check()
# analyze_cv_for_impact()
# ...

# --- GUI Fonksiyonları buraya gelecek ---
# Önce upload_file ve display_results'ı tanımlayın
def upload_file(job_desc):
    file_path = filedialog.askopenfilename(
        title="Bir CV Dosyası Seçin",
        filetypes=[("PDF Dosyaları", "*.pdf"), ("Word Dosyaları", "*.docx")]
    )
    if not file_path:
        return
    
    cv_text = extract_text_from_pdf(file_path)
    if "Hata" in cv_text:
        messagebox.showerror("Hata", cv_text)
        return

    keyword_analysis = calculate_keyword_score(cv_text, job_desc)
    advanced_suggestions = advanced_ats_check(cv_text)
    ai_analysis = analyze_cv_for_impact(cv_text)

    display_results(keyword_analysis, advanced_suggestions, ai_analysis)

def display_results(keyword_analysis, advanced_suggestions, ai_analysis):
    global result_text
    result_text.delete("1.0", tk.END)
    
    result_text.insert(tk.END, "### CV Analiz Sonuçları ###\n\n")
    result_text.insert(tk.END, f"Anahtar Kelime Uyumluluk Yüzdesi: {keyword_analysis['score_percentage']:.2f}%\n")
    result_text.insert(tk.END, "---------------------------------------\n")
    
    result_text.insert(tk.END, "Eşleşen Anahtar Kelimeler:\n")
    for keyword, count in keyword_analysis['matching_keywords'].items():
        result_text.insert(tk.END, f"- '{keyword}' : {count} kez bulundu\n")

    result_text.insert(tk.END, "\n### İyileştirme Önerileri ###\n")
        # Yeni eklenen kısım: Düşük puana göre uyarı
    # Yüzde 25'in altındaki bir puan için uyarı verelim. Bu değeri kendiniz belirleyebilirsiniz.
    if keyword_analysis['score_percentage'] < 25:
        result_text.insert(tk.END, "Uyumluluk yüzdesi oldukça düşük. İş ilanındaki temel beceri ve deneyim kelimelerini CV'nize eklemeye odaklanın.\n")
    
    # Eksik anahtar kelimeleri yazdır
    if missing_keywords:
        result_text.insert(tk.END, "Aşağıdaki anahtar kelimeler iş ilanında geçiyor ancak CV'nizde bulunamadı. Lütfen deneyimlerinizi yeniden yazarken bu kelimeleri eklemeye çalışın:\n")
        for keyword in missing_keywords:
            result_text.insert(tk.END, f"- '{keyword}'\n")

    all_suggestions = advanced_suggestions + ai_analysis['suggestions']
    if not all_suggestions:
        result_text.insert(tk.END, "Harika! CV'niz ATS kriterlerine oldukça uygun görünüyor.\n")
    else:
        for suggestion in all_suggestions:
            result_text.insert(tk.END, f"- {suggestion}\n")
            
    result_text.insert(tk.END, "\n### İstatistikler ###\n")
    result_text.insert(tk.END, f"Toplam İş İlanı Anahtar Kelimesi: {keyword_analysis['total_keywords_in_job_desc']}\n")
    result_text.insert(tk.END, f"CV'deki Toplam Eşleşme: {keyword_analysis['total_matches_in_cv']}\n")
    result_text.insert(tk.END, f"Etkili Eylem Kelimesi Sayısı: {ai_analysis['active_word_count']}\n")
    result_text.insert(tk.END, f"Nicel Veri Sayısı: {ai_analysis['quantitative_data_found']}\n")

# Son olarak, create_gui fonksiyonunu tanımlayın
def create_gui():
    global result_text
    root = tk.Tk()
    root.title("ATS-Dostu CV Analiz Aracı")
    root.geometry("800x600")
    
    input_frame = tk.Frame(root, padx=10, pady=10)
    input_frame.pack(fill="x")

    result_frame = tk.Frame(root, padx=10, pady=10)
    result_frame.pack(fill="both", expand=True)

    job_desc_label = tk.Label(input_frame, text="İş İlanı Metnini Buraya Yapıştırın:")
    job_desc_label.pack(side="left", padx=5)

    job_desc_text = scrolledtext.ScrolledText(input_frame, height=5, width=50)
    job_desc_text.pack(side="left", padx=5)

    upload_button = tk.Button(input_frame, text="CV Yükle", command=lambda: upload_file(job_desc_text.get("1.0", "end-1c")))
    upload_button.pack(side="left", padx=5)

    result_text = scrolledtext.ScrolledText(result_frame, wrap=tk.WORD, font=("Arial", 12))
    result_text.pack(fill="both", expand=True, padx=5, pady=5)
    
    root.mainloop()

# Uygulamayı başlat
if __name__ == "__main__":
    create_gui()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ilyda\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ilyda\AppData\Local\Temp\ipykernel_4496\3843126674.py", line 96, in <lambda>
    upload_button = tk.Button(input_frame, text="CV Yükle", command=lambda: upload_file(job_desc_text.get("1.0", "end-1c")))
                                                                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ilyda\AppData\Local\Temp\ipykernel_4496\3843126674.py", line 38, in upload_file
    display_results(keyword_analysis, advanced_suggestions, ai_analysis)
  File "C:\Users\ilyda\AppData\Local\Temp\ipykernel_4496\3843126674.py", line 59, in display_results
    if missing_keywords:
       ^^^^^^^^^^^^^^^^
NameError: name 'missing_keywords' is not defined


Last Step for CV Sugessions

In [45]:
def find_missing_keywords(job_desc_text, cv_text):
    """
    İş tanımında bulunan ancak CV'de eksik olan anahtar kelimeleri bulur.
    
    Args:
        job_desc_text (str): İş tanımı metni.
        cv_text (str): CV'den çıkarılan metin.
        
    Returns:
        list: Eksik anahtar kelimelerin listesi.
    """
    job_desc_text_lower = job_desc_text.lower()
    cv_text_lower = cv_text.lower()
    
    # İş ilanındaki anahtar kelimeleri bul (Önceki kodunuzdaki aynı regex'i kullanın)
    keywords = set(re.findall(r'\b[a-z0-9-]+\b|\b\w+\s\w+\b', job_desc_text_lower))
    
    missing_keywords = []
    
    for keyword in keywords:
        # Kelimenin CV'de olup olmadığını kontrol et
        if re.search(r'\b' + re.escape(keyword) + r'\b', cv_text_lower) is None:
            missing_keywords.append(keyword)
            
    return missing_keywords

# GUI'deki display_results fonksiyonunu güncelleyelim
# display_results() fonksiyonu içinde:
def display_results(keyword_analysis, advanced_suggestions, ai_analysis, cv_text, job_desc_text):
    # ... (diğer kodlar)

    result_text.insert(tk.END, "\n### İyileştirme Önerileri ###\n")
    
    all_suggestions = advanced_suggestions + ai_analysis['suggestions']
    
    # Eksik anahtar kelimeleri bul ve öneri olarak ekle
    missing_keywords = find_missing_keywords(job_desc_text, cv_text)
    if missing_keywords:
        result_text.insert(tk.END, "Aşağıdaki anahtar kelimeler iş ilanında geçiyor ancak CV'nizde bulunamadı. Lütfen deneyimlerinizi yeniden yazarken bu kelimeleri eklemeye çalışın:\n")
        for keyword in missing_keywords:
            result_text.insert(tk.END, f"- '{keyword}'\n")

    # Diğer önerileri ekle
    if not all_suggestions and not missing_keywords:
        result_text.insert(tk.END, "Harika! CV'niz ATS kriterlerine oldukça uygun görünüyor.\n")
    else:
        for suggestion in all_suggestions:
            result_text.insert(tk.END, f"- {suggestion}\n")
            
    # ... (diğer kodlar)

Intelligent Data Extraction: Ability to quickly and accurately extract text data from CVs in PDF and DOCX formats.

Keyword Matching Engine: Compares the keywords in the uploaded job posting with the CV text and calculates a critical match percentage for the applicant.

Advanced ATS Criteria Analysis: Doesn't just limit itself to keywords. It analyzes the structural integrity of the CV, the accuracy of section headings, and the effectiveness of the wording to ensure compliance with ATS (Applicant Tracking System) standards.

Intelligent Improvement Suggestions: In critical situations, such as a low keyword match score, it provides concrete feedback on CV deficiencies and areas for improvement. This helps users optimize their CVs specifically for the job posting.

User-Friendly Interface (GUI): Integrates the entire analysis engine into an intuitive and simple graphical interface that even non-technical users can easily use.